<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2025_sept/blob/main/4_pnl/contribuciones_estudiantes/recuperativa1510-ralitza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea recuperativa (15/10) modelo de lenguaje

*   Crea un modelo de lenguaje cargando un libro de internet, y que funcione con Gradio



In [1]:
import requests as rq
session = rq.Session()
headers={'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:143.0) Gecko/20100101 Firefox/143.0"}
session.headers.update({'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:143.0) Gecko/20100101 Firefox/143.0"})
session.get('https://httpbin.org/headers')

<Response [200]>

In [2]:
resultado=session.get("https://www.gutenberg.org/cache/epub/84/pg84-images.html",headers=headers)

# Preprocesamiento

En el lenguaje Html se deben extraer las lineas que empiecen con p.

In [3]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(resultado.text,"html.parser")
x=soup.find_all("p")
print(soup.p)
text=""
for i in range(len(x)):
    text+=x[i].text.strip()+"\n"
print(text[0:1000])

<p><strong>Title</strong>: Frankenstein; Or, The Modern Prometheus</p>
Title: Frankenstein; Or, The Modern Prometheus
Author: Mary Wollstonecraft Shelley
Release date: October 1, 1993 [eBook #84]
                Most recently updated: September 8, 2025
Language: English
Credits: Judith Boss, Christy Phillips, Lynn Hanninen and David Meltzer. HTML version by Al Haines.
        Further corrections by Menno de Leeuw.
To Mrs. Saville, England.
St. Petersburgh, Dec. 11th, 17—.
You will rejoice to hear that no disaster has accompanied the commencement of
an enterprise which you have regarded with such evil forebodings. I arrived
here yesterday, and my first task is to assure my dear sister of my welfare and
increasing confidence in the success of my undertaking.
I am already far north of London, and as I walk in the streets of Petersburgh,
I feel a cold northern breeze play upon my cheeks, which braces my nerves and
fills me with delight. Do you understand this feeling? This breeze, which ha

Se nota que en el lenguaje html el capitulo empieza tras nombrar todos los capitulos disponibles. Primero empieza con las Letters y luego inicia el capitulo 1.

*   Una forma de agilizar esto sera conocer previamente como inicia y termina cada extracto.

In [4]:
# Importar libreria regular expressions
import re

In [5]:
# Encuentra todas las ocurrencias del patrón "To Mrs. Saville, England."
# seguido de cualquier carácter hasta la siguiente ocurrencia del mismo patrón o el final del string.
letters = re.findall(r"To Mrs\. Saville, England\..*?(?=To Mrs\. Saville, England\.|\Z)", text, re.DOTALL)

# Almacena cada carta extraída en una variable llamada "letter_n"
for n, letter in enumerate(letters):
    globals()[f"letter_{n+1}"] = letter.strip()

print(len(letters))

4


De esto ultimo, son 4 cartas antes de iniciar los capitulos. Como el texto pierde las palabras "Chapter 1" ,... El resto de capitulos se debe extraer sabiendo como inicia y como finalizan o dar mejores indicaciones a Beautifulsoup.

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(resultado.text, "html.parser")

improved_text = ""

# Encontrar todas las etiquetas potencialmente de contenido
content_tags = soup.find_all(['p', 'h2', 'h3', 'h4', 'h5', 'h6'])

for tag in content_tags:
    text = tag.get_text().strip()
    # Verificar si el texto parece un capítulo o título de carta
    if "Chapter" in text or "Letter" in text or "CONTENTS" in text or "Title" in text or "Author" in text or "Release date" in text or "Language" in text or "Credits" in text:
        improved_text += f"\n\n--- {text} ---\n\n" # Añadir marcadores prominentes para títulos
    else:
        improved_text += text + " " # Añadir otro texto con un espacio

# Imprimir los primeros 2000 caracteres del texto mejorado para ver la estructura
print(improved_text[:2000])

The Project Gutenberg eBook of Frankenstein; Or, The Modern Prometheus 

--- Title: Frankenstein; Or, The Modern Prometheus ---



--- Author: Mary Wollstonecraft Shelley ---



--- Release date: October 1, 1993 [eBook #84]
                Most recently updated: September 8, 2025 ---



--- Language: English ---



--- Credits: Judith Boss, Christy Phillips, Lynn Hanninen and David Meltzer. HTML version by Al Haines.
        Further corrections by Menno de Leeuw. ---

or, the Modern Prometheus by Mary Wollstonecraft (Godwin) Shelley 

--- CONTENTS ---



--- Letter 1 ---

To Mrs. Saville, England. St. Petersburgh, Dec. 11th, 17—. You will rejoice to hear that no disaster has accompanied the commencement of
an enterprise which you have regarded with such evil forebodings. I arrived
here yesterday, and my first task is to assure my dear sister of my welfare and
increasing confidence in the success of my undertaking. I am already far north of London, and as I walk in the streets of Peters

Con este texto mejorado se utilizara "improved text".

# Modelo de lenguaje N-Gram

In [7]:
import nltk
from numpy.random import randint, seed
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
my_text = [improved_text.lower()]  # Convertir a minúsculas y crear una lista. => Requerido por CountVectorizer().

In [9]:
n = 3 # Puede cambiarse a un número igual o mayor que 2.
n_min = n
n_max = n
n_gram_type = 'word' # n-Grama con palabras.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [10]:
n_grams = vectorizer.fit(my_text).get_feature_names_out()        # Obtener los n-gramas como una lista.
n_gram_cts = vectorizer.transform(my_text).toarray()             # La salida es un array de arrays.
n_gram_cts = list(n_gram_cts[0])                                 # Convertir en una lista simple.

In [11]:
# Convertir enteros NumPy a enteros estándar de Python antes de combinar y mostrar
n_gram_cts_int = [int(count) for count in n_gram_cts]
list(zip(n_grams, n_gram_cts_int))  # Crear una lista de tuplas y mostrar.

[('10 spent the', 1),
 ('11 it is', 1),
 ('11th 17 you', 1),
 ('11th the passage', 1),
 ('12 lay on', 1),
 ('12th 17 clerval', 1),
 ('12th it is', 1),
 ('13 now hasten', 1),
 ('13th 17 my', 1),
 ('14 some time', 1),
 ('15 such was', 1),
 ('16 cursed cursed', 1),
 ('17 clerval who', 1),
 ('17 dear dear', 1),
 ('17 how slowly', 1),
 ('17 my affection', 1),
 ('17 my dear', 1),
 ('17 so strange', 1),
 ('17 the being', 1),
 ('17 this letter', 1),
 ('17 yesterday the', 1),
 ('17 you have', 1),
 ('17 you will', 1),
 ('18 day after', 1),
 ('18th 17 dear', 1),
 ('18th 17 this', 1),
 ('19 london was', 1),
 ('1993 ebook 84', 1),
 ('19th 17 yesterday', 1),
 ('20 sat one', 1),
 ('2025 language english', 1),
 ('21 was soon', 1),
 ('22 the voyage', 1),
 ('23 it was', 1),
 ('24 my present', 1),
 ('26th 17 you', 1),
 ('27th of march', 1),
 ('28th march 17', 1),
 ('2d write to', 1),
 ('31st we were', 1),
 ('5th 17 so', 1),
 ('5th scene has', 1),
 ('7th 17 my', 1),
 ('7th my niece', 1),
 ('7th the die', 

In [12]:
# Verificar si hay n-gramas con conteo cero
zero_count_ngrams = [ngram for ngram, count in zip(n_grams, n_gram_cts_int) if count <= 0]

if zero_count_ngrams:
    print("Los siguientes n-gramas tienen conteo cero o menor:")
    for ngram in zero_count_ngrams:
        print(f"- {ngram}")
else:
    print("Todos los n-gramas tienen conteo mayor a cero.")

Todos los n-gramas tienen conteo mayor a cero.


In [28]:
# Encontrar claves en my_dict con más de 2 palabras siguientes predichas
phrases_with_multiple_predictions = {key: value for key, value in my_dict.items() if len(value) > 2}

# Mostrar las frases y sus posibles palabras siguientes
if phrases_with_multiple_predictions:
    print("Frases con más de 2 palabras siguientes predichas:")
    for phrase, predictions in phrases_with_multiple_predictions.items():
        print(f"- '{phrase}': {predictions}")
else:
    print("No se encontraron frases con más de 2 palabras siguientes predichas.")

Frases con más de 2 palabras siguientes predichas:
- 'abhor me': ['and', 'what', 'will']
- 'able to': ['decipher', 'hold', 'leave', 'perform', 'pronounce', 'satisfy', 'secure', 'speak']
- 'about five': ['and', 'in', 'miles', 'years']
- 'about the': ['commission', 'end', 'gentleman', 'isle', 'middle', 'time', 'world']
- 'about to': ['explore', 'fail', 'form', 'mock', 'perish', 'proceed', 'relate', 'remove', 'sink', 'undertake']
- 'about twenty': ['leagues', 'words', 'yards']
- 'about two': ['clock', 'hours', 'miles']
- 'above all': ['and', 'as', 'desired', 'the']
- 'above the': ['dome', 'fiend', 'wretched']
- 'absence of': ['his', 'summer', 'the']
- 'accompanied by': ['country', 'me', 'moderate', 'the']
- 'accompany me': ['and', 'not', 'to']
- 'account of': ['my', 'myself', 'the', 'your', 'yourself']
- 'accounts of': ['domestic', 'the', 'vice']
- 'accustomed to': ['do', 'grand', 'see', 'the']
- 'acquainted with': ['any', 'him', 'it', 'its', 'more', 'my', 'other', 'the', 'their']
- 'acro

Ahora, se entrenara el modelo mediante un diccionario de los n-gramas.

In [14]:
n = 3     # Puede cambiarse a un número igual o mayor que 2.
n_min = n
n_max = n
n_gram_type = 'word' # n-Grama con palabras
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [15]:
import nltk
nltk.download('punkt_tab') # para la tokenizacion se agrega

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [16]:
n_grams = vectorizer.fit(my_text).get_feature_names_out()  # Una lista de n-Gramas.
my_dict = {}
for a_gram in n_grams:
    words = nltk.word_tokenize(a_gram)
    a_nm1_gram = ' '.join(words[0:n-1])  # (n-1)-Grama.
    next_word = words[-1]  # Palabra después del a_nm1_gram.
    if a_nm1_gram not in my_dict.keys():
        my_dict[a_nm1_gram] = [next_word] # a_nm1_gram es una nueva clave. Inicializar la entrada del diccionario.
    else:
        my_dict[a_nm1_gram] += [next_word] # a_nm1_gram ya está en el diccionario. Añadir la palabra.

In [17]:
# Ver el diccionario.
my_dict

{'10 spent': ['the'],
 '11 it': ['is'],
 '11th 17': ['you'],
 '11th the': ['passage'],
 '12 lay': ['on'],
 '12th 17': ['clerval'],
 '12th it': ['is'],
 '13 now': ['hasten'],
 '13th 17': ['my'],
 '14 some': ['time'],
 '15 such': ['was'],
 '16 cursed': ['cursed'],
 '17 clerval': ['who'],
 '17 dear': ['dear'],
 '17 how': ['slowly'],
 '17 my': ['affection', 'dear'],
 '17 so': ['strange'],
 '17 the': ['being'],
 '17 this': ['letter'],
 '17 yesterday': ['the'],
 '17 you': ['have', 'will'],
 '18 day': ['after'],
 '18th 17': ['dear', 'this'],
 '19 london': ['was'],
 '1993 ebook': ['84'],
 '19th 17': ['yesterday'],
 '20 sat': ['one'],
 '2025 language': ['english'],
 '21 was': ['soon'],
 '22 the': ['voyage'],
 '23 it': ['was'],
 '24 my': ['present'],
 '26th 17': ['you'],
 '27th of': ['march'],
 '28th march': ['17'],
 '2d write': ['to'],
 '31st we': ['were'],
 '5th 17': ['so'],
 '5th scene': ['has'],
 '7th 17': ['my'],
 '7th my': ['niece'],
 '7th the': ['die'],
 '84 most': ['recently'],
 '9th the

El modelo se entreno para predecir la palabra siguiente a las expresiones mostradas. Por ejemplo, "adress the" predice despues "court" segun el texto entregado.

In [18]:
print(len(my_dict[a_nm1_gram]))

1


In [19]:
# Función auxiliar que elige la palabra siguiente.
def predict_next(a_nm1_gram):
    value_list_size = len(my_dict[a_nm1_gram]) # longitud del valor correspondiente a la clave = a_nm1_gram.
    i_pick = randint(0, value_list_size) # Un número aleatorio del rango 0 ~ (value_list_size).
    return my_dict[a_nm1_gram][i_pick] # Retornar la palabra siguiente elegida aleatoriamente.

In [20]:
# Probar.
input_str = 'address the'  # ¡Tiene que ser un n-Grama VÁLIDO!
predict_next(input_str)

'court'

In [21]:
# Repetir
input_str = 'you are' # Debe estar en el diccionario
for i in range(6):
    print(predict_next(input_str))

younger
my
of
too
getting
mistaken


Ahora para predecir una secuencia

In [22]:
# Inicializar la semilla aleatoria.
seed(123)
my_seed_str = 'you are'

In [23]:
a_nm1_gram = my_seed_str
output_string = my_seed_str  # Inicializar la cadena de salida.
while a_nm1_gram in my_dict:
    output_string += " " + predict_next(a_nm1_gram)
    words = nltk.word_tokenize(output_string)
    a_nm1_gram = ' '.join(words[-n+1:])  # Actualizar a_nm1_gram.

In [24]:
output_string

'you are bound you may deduce an apt moral from my inmost soul justine started when she had passed sleepless night and all will be blessing to them been for ever but soon he cried by the enchanting appearance of hilarity that brought misery on us all ay stare if you saw him the greatest ardour for virtue rise within me desire to learn the hidden laws of electricity on this wonderful man sometimes have endeavoured to put an end for ever that the wind favourable all smiled on our way to the milk of one cow which gave inexpressible grace to his destruction was mine during the day of expectation delayed fills them with pleasure have destroyed my friends he devoted to it he quitted his country nor returned until he was entirely confined to hunger thirst and want felix carried with pleasure my country which when was happy and prosperous guarded by the modern professors of natural history and misfortunes on every point of general literature he displays unbounded knowledge and how listened to 

La prediccion de "you are" sigue despues una secuencia de texto que no tiene mucha coherencia.

In [25]:
# Inicializar la semilla aleatoria.
seed(123)
my_seed_str = 'you will'
a_nm1_gram = my_seed_str
output_string = my_seed_str  # Inicializar la cadena de salida.
while a_nm1_gram in my_dict:
    output_string += " " + predict_next(a_nm1_gram)
    words = nltk.word_tokenize(output_string)
    a_nm1_gram = ' '.join(words[-n+1:])  # Actualizar a_nm1_gram.

In [26]:
output_string

'you will be born to replace those of drunken man fell at last in state of your mother this picture was miniature of william justine and indeed any animal endued with life do your duty towards me as their pest whose selfishness had not broken until near midnight and the moon arose and as an insuperable barrier between me and although not so wretched that the lives of all excellence and endeavoured by arguments deduced from the wreck of his family he could judge by the restless waves that were accumulated around him within whose circle no grief or folly ventures will you be so kind as to destroy me wantonly did the murderer grasp how much more murdered that could lead to different conclusion between two and three in the heavens cloudless and her cheek pale but there was one which have enjoyed for several hours this sudden certainty of life chapter 18 day after our union we should proceed to villa lavenza and spend his life in my dying moments shall not curse my maker his words had stran

# Testeo con Gradio

In [27]:
import gradio as gr
import nltk
from numpy.random import randint, seed
import random # Importar el módulo random

# Asegurarse de tener los datos necesarios de NLTK descargados
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


# Asumiendo que my_dict y predict_next ya están definidos en celdas anteriores
def generate_text(start_phrase, num_words=50):
    """Genera texto basado en el modelo de N-gramas."""
    seed(123) # Puedes cambiar la semilla para diferentes secuencias aleatorias
    output_string = start_phrase
    a_nm1_gram = start_phrase.lower() # Usar minúsculas para búsqueda en el diccionario

    # Tokenizar la frase inicial para obtener las últimas n-1 palabras
    words = nltk.word_tokenize(a_nm1_gram)
    # Asegurar que tenemos al menos n-1 palabras para empezar
    if len(words) < n - 1:
        return "Por favor ingresa una frase inicial con al menos {} palabras.".format(n - 1)

    a_nm1_gram = ' '.join(words[-n+1:])


    for _ in range(num_words):
        if a_nm1_gram in my_dict:
            next_word = predict_next(a_nm1_gram)
            output_string += " " + next_word
            words = nltk.word_tokenize(output_string)
            a_nm1_gram = ' '.join(words[-n+1:])
        else:
            break # Detenerse si el n-grama no está en el diccionario

    return output_string

# Obtener una lista de claves del diccionario para usar como ejemplos
dict_keys = list(my_dict.keys())
# Seleccionar algunas claves aleatorias como ejemplos, asegurando que haya suficientes claves
num_examples = min(5, len(dict_keys)) # Obtener máximo 5 ejemplos
# Crear una lista de listas para ejemplos, cada lista interna corresponde a los inputs de la función
examples_list = [[phrase] for phrase in random.sample(dict_keys, num_examples)]


# Crear la interfaz de Gradio
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label=f"Ingresa una frase inicial (al menos {n-1} palabras):"),
        gr.Slider(minimum=10, maximum=200, value=50, step=10, label="Número de palabras a generar:")
        ],
    outputs="text",
    title="Generador de Texto con N-Gramas de Frankenstein",
    description="Genera texto basado en Frankenstein de Mary Shelley usando un modelo de N-gramas.",
    examples=examples_list # Añadir ejemplos aquí a nivel de Interface
)

# Lanzar la aplicación Gradio
iface.launch()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33e7eef7c7fd5415e8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
